In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.optimizers import Adam, SGD
import random

from keras.layers import Flatten, Dropout, BatchNormalization, Reshape, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Input
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler

from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import pandas as pd

from keras.preprocessing import image

from sklearn.model_selection import train_test_split

import os
import shutil

import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
# total_images = 27558
# validation_size = 0.20

# training_parasitized_folder_name = "training/Parasitized"
# training_uninfected_folder_name = "training/Uninfected"

# validation_parasitized_folder_name = "validation/Parasitized"
# validation_uninfected_folder_name = "validation/Uninfected"

In [ ]:
# parasitized_folder = os.listdir("training/Parasitized")
# uninfected_folder = os.listdir("training/Uninfected")

# validation_folder_size = int(len(parasitized_folder) * 0.20)

# for image_name in parasitized_folder[:validation_folder_size]:
#     image_url = f'{training_parasitized_folder_name}/{image_name}'
#     shutil.move(image_url, validation_parasitized_folder_name)
  
# for image_name in uninfected_folder[:validation_folder_size]:
#     image_url = f'{training_uninfected_folder_name}/{image_name}'
#     shutil.move(image_url, validation_uninfected_folder_name)

In [ ]:
# def plot_images(image_index, folder_name, image_name, title):
#   sp = figure.add_subplot(2, 3, image_index + 1)
#   sp.axis('Off')
#   image_path = f'{folder_name}/{image_name}'
#   image_loaded = img=mpimg.imread(image_path)
#   sp.set_title(title, fontsize=16)
#   plt.imshow(image_loaded, interpolation=None)

# figure = plt.figure(figsize=(12, 6))
# parasitized_images = os.listdir("training/Parasitized")[0:3]
# uninfected_images = os.listdir("training/Uninfected")[0:3]

# for image_index, image_name in enumerate(parasitized_images):
#   plot_images(image_index, training_parasitized_folder_name, image_name, "Parasitized")

# for image_index, image_name in enumerate(uninfected_images):
#   plot_images(image_index + 3, training_uninfected_folder_name, image_name, "Uninfected")

In [ ]:
batch_size = 32
img_size = 96
input_img_size = (96, 96, 3)
num_classes = 2
total_images = 27558

In [ ]:
 datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest",
     validation_split=0.2)
train_generator = datagen.flow_from_directory(
    "../input/cell-images-for-detecting-malaria/cell_images/cell_images/",
    target_size=(img_size, img_size),
    color_mode="rgb",
	  shuffle=True,
    batch_size=batch_size, subset='training')


val_generator = datagen.flow_from_directory(
    "../input/cell-images-for-detecting-malaria/cell_images/cell_images/",
    target_size=(img_size, img_size),
    color_mode="rgb",
	  shuffle=False,
    batch_size=batch_size,subset='validation')

In [ ]:
val_generator

In [ ]:
train_steps = int((total_images - (total_images//2*0.2 * 2)) // batch_size)
val_steps = int((total_images//2*0.2 * 2) // batch_size) 

## Model

In [ ]:
def create_model():
  input_tensor = Input(shape=input_img_size)
  
  mobile_model = MobileNetV2(
       weights=None,
       input_tensor=input_tensor,
       input_shape=input_img_size,
       alpha=1.5,
       include_top=False)
  
  for layer in mobile_model.layers:
    layer.trainable = True
  
  mobile_model_output = mobile_model.output
  classification_layer = Flatten()(mobile_model_output)
  classification_layer = Dense(256, activation='relu')(classification_layer)
  classification_layer = Dropout(0.5)(classification_layer)
  predictions = Dense(activation="softmax", units=num_classes)(classification_layer)

  model = Model(inputs=input_tensor, outputs=predictions)
  
  return model

In [ ]:
learning_rate = 3e-4
epochs = 30

In [ ]:
def polynomial_decay(epoch):
	power = 1.0
 
	alpha = learning_rate * (1 - (epoch / float(epochs))) ** power
	return alpha

In [ ]:
weights_name = "epoch={epoch:02d}|accuracy={val_acc:.4f}.h5"

checkpoint = ModelCheckpoint(weights_name, monitor="val_acc", verbose=1, save_best_only=True,
                                 save_weights_only=True, mode="max", period=1)

lr_decay = LearningRateScheduler(polynomial_decay)

optimizer = SGD(lr=learning_rate, momentum=0.9)

In [ ]:
model = create_model()
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])

In [ ]:
trained_model = model.fit_generator(train_generator,
                        epochs=epochs,
                        steps_per_epoch=train_steps,
#                         batch_size = 64,
                        callbacks=[checkpoint, lr_decay],
                        validation_data=val_generator,
                        validation_steps=val_steps,
                        verbose=1)

In [ ]:
def plot_validation_training(metric, trained_model):
  validation_metric = trained_model.history[f'val_{metric}']
  training_metric = trained_model.history[metric]
  epochs = range(len(training_metric))
  plt.plot(epochs, training_metric, 'b', label=f'Training {metric}')
  plt.plot(epochs, validation_metric, 'r', label=f'Validation {metric}')
  plt.ylim(bottom=0)
  plt.xlabel('Epochs ', fontsize=16)
  plt.ylabel(metric, fontsize=16)
  loc = 'upper right' if metric == "loss" else 'lower right'
  plt.legend(loc=loc)
  plt.title(f'Training and validation {metric}', fontsize = 20)
  plt.show()

In [ ]:
plot_validation_training("loss", trained_model)

In [ ]:
plot_validation_training("acc", trained_model)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
model.load_weights("epoch=24_accuracy=0.9646.h5")

In [ ]:
val_generator.reset()

In [ ]:
classes_predicted = model.predict_generator(val_generator, steps=val_steps, verbose=1)

In [ ]:
len(classes_predicted)

In [ ]:
real_classes = np.argmax(classes_predicted, axis=1)

In [ ]:
val_labels = val_generator.classes

In [ ]:
val_labels = val_labels[:5504]

In [ ]:
len(val_labels), len(real_classes)

In [ ]:
val_generator.class_indices

In [ ]:
classes_names = ["Parasitized", "Uninfected"]

In [ ]:
cm = confusion_matrix(val_labels, real_classes, labels=range(num_classes))
plot_confusion_matrix(cm, classes_names)

In [ ]:
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
print(f'sensitivity: {sensitivity}')

specificity = cm[1, 1] / (cm[1, 1] + cm[1, 0])
print(f'specifity: {specificity}')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(val_labels, real_classes, target_names=classes_names)
print(report)

## Gradient maps

In [ ]:
!pip uninstall keras-vis

In [ ]:
!pip install git+https://github.com/raghakot/keras-vis.git -U

In [ ]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations

In [ ]:
parasitized_folder = os.listdir("training/Parasitized")
uninfected_folder = os.listdir("training/Uninfected")

In [ ]:
parasitized_image_path = f'{training_parasitized_folder_name}/{parasitized_folder[20]}'
uninfected_image_path = f'{training_uninfected_folder_name}/{uninfected_folder[20]}'

parasitized_image = utils.load_img(parasitized_image_path, target_size=(96, 96))
uninfected_image = utils.load_img(uninfected_image_path, target_size=(96, 96))

In [ ]:
parasitized_image_model = image.load_img(parasitized_image_path, target_size=(96, 96))
uninfected_image_model = image.load_img(uninfected_image_path, target_size=(96, 96))

parasitized_image_model = image.img_to_array(parasitized_image)
parasitized_image_model = preprocess_input(parasitized_image)

uninfected_image_model = image.img_to_array(uninfected_image)
uninfected_image_model = preprocess_input(uninfected_image)

preprocessed_images = np.array([parasitized_image_model, uninfected_image_model])

In [ ]:
predictions = model.predict(preprocessed_images)

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
images_array = [parasitized_image, uninfected_image]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 8))
fig.suptitle('Normal Images')

for index, img in enumerate(images_array):   
  class_index = predicted_classes[index]
  ax[index].set_title(f'class: {classes_names[index]}, pred: {classes_names[class_index]}')
  ax[index].imshow(img)

In [ ]:
layer_idx = utils.find_layer_idx(model, 'dense_2')

fig, ax = plt.subplots(1, 2, figsize=(8, 8))
fig.suptitle('Gradient maps')

for index, img in enumerate(images_array):    
    grads = visualize_saliency(model, layer_idx, filter_indices=index, seed_input=img)
    ax[index].set_title(classes_names[index])
    ax[index].imshow(grads, cmap='jet')

In [ ]:
for modifier in ['guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle(modifier)
    
    for index, img in enumerate(images_array):    
        grads = visualize_saliency(model, layer_idx, filter_indices=index, 
                                   seed_input=img, backprop_modifier=modifier)   
        ax[index].imshow(grads, cmap='jet')

In [ ]:
import matplotlib.cm as cm
from vis.visualization import visualize_cam

penultimate_layer = utils.find_layer_idx(model, 'Conv_1_bn')

for modifier in [None, 'guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 2)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for index, img in enumerate(images_array):    
        grads = visualize_cam(model, layer_idx, filter_indices=index, 
                              seed_input=img, penultimate_layer_idx=penultimate_layer,
                              backprop_modifier=modifier)        
  
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        ax[index].imshow(overlay(jet_heatmap, img))

## Save model

In [ ]:
model_json = model.to_json()
with open("malaria_model.json", "w") as json_file:
    json_file.write(model_json)